# Named Entity Recognition for the decicontas.br dataset

In [5]:
import re
import ast
import json
import pprint

import pandas as pd

from typing import List, Tuple, Dict
from langchain_openai import  AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.prompts import StringPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sklearn.metrics import precision_score, recall_score, f1_score
from dotenv import load_dotenv

from tools.schema import (
    NERDecisao
)


load_dotenv()

gpt4o = AzureChatOpenAI(
    deployment_name="gpt-4o",  # deployment com modelo gpt-4o
    model_name="gpt-4o",       # opcional para compatibilidade
)

gpt35 = AzureChatOpenAI(
    deployment_name="gpt-35",  # outro deployment, com gpt-35-turbo
    model_name="gpt-35-turbo",
)

gpt4turbo = AzureChatOpenAI(
    deployment_name="gpt-4-turbo",
    model_name="gpt-4",
)

extractor_gpt4o = gpt4o.with_structured_output(NERDecisao, include_raw=False, method="function_calling")
extractor_gpt35 = gpt35.with_structured_output(NERDecisao, include_raw=False, method="function_calling")
extractor_gpt4turbo = gpt4turbo.with_structured_output(NERDecisao, include_raw=False, method="function_calling")

# Example use

In [2]:
from tools.prompt import generate_few_shot_ner_prompts

from tools.schema import NERDecisao

In [ ]:
EXAMPLE_TEXT = '''
DECIDEM os Conselheiros do Tribunal de Contas do Estado, à unanimidade, em consonância com a informação do Corpo Técnico e com o parecer do Ministério Público que atua junto a esta Corte de Contas, acolhendo integralmente o voto do Conselheiro Relator, julgar: a) pela DENEGAÇÃO DE REGISTRO ao ato concessivo da aposentadoria e à despesa dele decorrente; b) pela determinação ao IPERN, à vista da Lei Complementar Estadual nº 547/2015, para que, no prazo de 60 (sessenta) dias, após o trânsito em julgado desta decisão, adote as correções necessárias para regularização do ato concessório, do cálculo dos proventos e de sua respectiva implantação; c) no caso de descumprimento da presente decisão, a responsabilização do titular da pasta responsável por seu atendimento, sem prejuízo da multa cominatória desde já fixada no valor de R$ 50,00 (cinquenta reais) por dia que superar o interregno fixado no item `b`, com base no art. 110 da Lei Complementar Estadual nº 464/2012, valor este passível de revisão e limitado ao teto previsto no art. 323, inciso II, alínea `f`, do Regimento Interno, a ser apurado por ocasião de eventual subsistência de mora.
'''
prompt_with_few_shot = generate_few_shot_ner_prompts(EXAMPLE_TEXT)


ChatPromptValue(messages=[SystemMessage(content='Você é um especialista em extração de entidades nomeadas com precisão excepcional. Sua tarefa é identificar e extrair informações específicas do texto fornecido, seguindo estas diretrizes:\n\n1. Extraia as informações exatamente como aparecem no texto, sem interpretações ou alterações.\n2. Se uma informação solicitada não estiver presente ou for ambígua, retorne null para esse campo.\n3. Mantenha-se estritamente dentro do escopo das entidades e atributos definidos no esquema fornecido.\n4. Preste atenção especial para manter a mesma ortografia, pontuação e formatação das informações extraídas.\n5. Não infira ou adicione informações que não estejam explicitamente presentes no texto.\n6. Se houver múltiplas menções da mesma entidade, extraia todas as ocorrências relevantes.\n7. Ignore informações irrelevantes ou fora do contexto das entidades solicitadas.\n\nLembre-se: sua precisão e aderência ao texto original são cruciais para o sucesso 

In [4]:
prompt_with_few_shot.messages

[SystemMessage(content='Você é um especialista em extração de entidades nomeadas com precisão excepcional. Sua tarefa é identificar e extrair informações específicas do texto fornecido, seguindo estas diretrizes:\n\n1. Extraia as informações exatamente como aparecem no texto, sem interpretações ou alterações.\n2. Se uma informação solicitada não estiver presente ou for ambígua, retorne null para esse campo.\n3. Mantenha-se estritamente dentro do escopo das entidades e atributos definidos no esquema fornecido.\n4. Preste atenção especial para manter a mesma ortografia, pontuação e formatação das informações extraídas.\n5. Não infira ou adicione informações que não estejam explicitamente presentes no texto.\n6. Se houver múltiplas menções da mesma entidade, extraia todas as ocorrências relevantes.\n7. Ignore informações irrelevantes ou fora do contexto das entidades solicitadas.\n\nLembre-se: sua precisão e aderência ao texto original são cruciais para o sucesso desta tarefa.', additiona

In [6]:
result = extractor_gpt4o.invoke(prompt_with_few_shot)

In [7]:
result.model_dump()

{'multas_fixas': [],
 'multas_percentuais': [],
 'obrigacoes_multa': [{'descricao_obrigacaomulta': 'determinação ao IPERN, à vista da Lei Complementar Estadual nº 547/2015, para que, no prazo de 60 (sessenta) dias, após o trânsito em julgado desta decisão, adote as correções necessárias para regularização do ato concessório, do cálculo dos proventos e de sua respectiva implantação; c) no caso de descumprimento da presente decisão, a responsabilização do titular da pasta responsável por seu atendimento, sem prejuízo da multa cominatória desde já fixada no valor de R$ 50,00 (cinquenta reais) por dia que superar o interregno fixado no item `b`, com base no art. 110 da Lei Complementar Estadual nº 464/2012, valor este passível de revisão e limitado ao teto previsto no art. 323, inciso II, alínea `f`, do Regimento Interno, a ser apurado por ocasião de eventual subsistência de mora.'}],
 'ressarcimentos': [],
 'obrigacoes': [],
 'recomendacoes': []}

# Loading and setting up

In [8]:
df_fewshot = pd.read_json("dataset/labeled_data/fewshot_decicontas.json")
fewshot_json = df_fewshot[['annotations', 'data']]

df_decicontas = pd.read_json("dataset/labeled_data/decicontas.json")
decicontas_json = df_decicontas[['annotations', 'data']]


# Testing with golden labels

In [ ]:
golden_models = []
for index, row in decicontas_json.iterrows():
    if index % 10 == 0 and index > 0:
        print(f"Processando instância {index} de {len(fewshot_json)}")
    prompt_with_few_shot = generate_few_shot_ner_prompts(row['data']['text'])
    result = extractor_gpt4o.invoke(prompt_with_few_shot)
    golden_models.append(
        {
            'index': index,
            'text': row['data']['text'], 
            'pred': result.model_dump(), 
            'golden': [r['value'] for r in row['annotations'][0]['result']],
            'model': 'gpt-4o'
        }
    )

Processando instância 10 de 12


In [23]:
df_golden_models = pd.DataFrame(golden_models)
df_golden_models.to_json("dataset/labeled_data/golden_models_decicontas.json", orient="records", force_ascii=False, indent=2)

In [25]:
df_golden_models.to_latex()

"\\begin{tabular}{lrllll}\n\\toprule\n & index & text & pred & golden & model \\\\\n\\midrule\n0 & 0 &                                 DECIDEM os Conselheiros do Tribunal de Contas do Estado, à unanimidade, em consonância com a informação do Corpo Técnico e com o parecer do Ministério Público que atua junto a esta Corte de Contas, acolhendo integralmente o voto do Conselheiro Relator, julgar: a) pela DENEGAÇÃO DE REGISTRO ao ato concessivo da aposentadoria e à despesa dele decorrente; b) pela determinação ao IPERN, à vista da Lei Complementar Estadual nº 547/2015, para que, no prazo de 60 (sessenta) dias, após o trânsito em julgado desta decisão, adote as correções necessárias para regularização do ato concessório, do cálculo dos proventos e de sua respectiva implantação; c) no caso de descumprimento da presente decisão, a responsabilização do titular da pasta responsável por seu atendimento, sem prejuízo da multa cominatória desde já fixada no valor de R$ 50,00 (cinquenta reais) por d

In [ ]:
# criar métricas por label
# usar seqeval
# métrica IoU intersection over union (IoU)

In [ ]:
row = fewshot_json.iloc[0]

# Extracting pydantic entities

In [5]:
from tools.prompt import generate_few_shot_ner_prompts
from langchain_core.runnables import RunnableLambda, RunnableMap

prepare_prompt_chain = RunnableLambda(lambda x: generate_few_shot_ner_prompts(x["text"]))
ner_chain = prepare_prompt_chain | gpt4o



In [6]:
exemplo_erro_1 = decicontas_json.loc[176]['data']['text']
exemplo_erro_2 = decicontas_json.loc[177]['data']['text']
exemplo_erro_3 = decicontas_json.loc[1280]['data']['text']
exemplo_erro_4 = decicontas_json.loc[1336]['data']['text']

In [11]:
result = ner_chain.invoke({"text": exemplo_erro_1})
pprint.pprint(exemplo_erro_1)
pprint.pprint(result.content)

('Vistos, relatados e discutidos estes autos,acatando o entendimento do '
 'Ministério Público Especial, com fulcro nos fundamentos jurídicos dantes '
 'explanados, ACORDAM os Conselheiros, nos termos do voto proferido pelo '
 'Conselheiro Relator, julgar: \n'
 '\n'
 'a)\tpela APLICAÇÃO DE MULTA no valor de R$1.000,00 (mil reais) para o então '
 'gestor responsável, à época dos fatos, pelo Instituto de Previdência dos '
 'Servidores do Estado do Rio Grande do Norte - IPERN,  senhor Nereu Batista '
 'Linhares, nos termos do artigo 107, inciso II, alínea “f”, da Lei '
 'Complementar Estadual nº 464/2012 c/c o artigo 323, inciso II, alínea `f`, '
 'do Novel Regimento Interno desta Casa, em virtude do descumprimento de '
 'determinação do Tribunal (Decisão nº 1255/2020-TC).\n'
 ' \n'
 'b)\tpela INTIMAÇÃO da referida autoridade competente nominada, a fim de que '
 'tome conhecimento desta decisão e, se for o caso, apresente recurso no prazo '
 'regimental. \n'
 '\n'
 'c)\tpela RENOVAÇÃO DA 

In [ ]:
result = ner_chain.invoke({"text": exemplo_erro_2})
pprint.pprint(exemplo_erro_2)
pprint.pprint(result.content)

('Vistos, relatados e discutidos estes autos, acatando o entendimento do '
 'Ministério Público Especial, com fulcro nos fundamentos jurídicos dantes '
 'explanados, ACORDAM os Conselheiros, nos termos do voto proferido pelo '
 'Conselheiro Relator, julgar:\n'
 'a)\tpela APLICAÇÃO DE MULTA no valor de R$1.000,00 (mil reais) para o então '
 'gestor responsável, à época dos fatos, pelo Instituto de Previdência dos '
 'Servidores do Estado do Rio Grande do Norte - IPERN,  senhor Nereu Batista '
 'Linhares, nos termos do artigo 107, inciso II, alínea “f”, da Lei '
 'Complementar Estadual nº 464/2012 c/c o artigo 323, inciso II, alínea `f`, '
 'do Novel Regimento Interno desta Casa, em virtude do descumprimento de '
 'determinação do Tribunal (Decisão nº 917/2020-TC).\n'
 'b)\tpela INTIMAÇÃO da referida autoridade competente nominada, a fim de que '
 'tome conhecimento desta decisão e, se for o caso, apresente recurso no prazo '
 'regimental. \n'
 'c)\tpela RENOVAÇÃO DA DETERMINAÇÃO constan

In [8]:
result = ner_chain.invoke({"text": exemplo_erro_3})
pprint.pprint(exemplo_erro_3)
pprint.pprint(result)

('Vistos, relatados e discutidos estes autos, ACORDAM os(as) Conselheiros(as), '
 'nos termos do voto proferido pelo Conselheiro Relator, em consonância com o '
 'Corpo Técnico da DAM e com o parecer do Ministério Público junto a esta '
 'Corte, julgar no sentido de conhecer e prover o Pedido de Reconsideração '
 '006990/2019-TC – (evento 41) interposto por Kerginaldo Medeiros de Araújo, '
 'para reformar, integralmente, o Acórdão nº 257/2019-TC-2ª Câmara (evento '
 '30), com vistas a:\n'
 '1) Declarar a prescrição da pretensão punitiva em favor do recorrente;\n'
 '2) Determinar o arquivamento do processo após o trânsito em julgado do '
 'Acórdão;\n'
 '3) Recomendar à Secretaria de Controle Externo – SECEX deste Tribunal de '
 'Contas a adoção de procedimentos com vistas a impedir que processos fiquem '
 'paralisados por mais de três anos nas Unidades Técnicas a ela vinculadas, '
 'para que se evite, assim, a ocorrência de prescrição intercorrente, como no '
 'presente caso;\n'
 '4) Sa

In [9]:
result = ner_chain.invoke({"text": exemplo_erro_4})
pprint.pprint(exemplo_erro_4)
pprint.pprint(result)

('DECIDEM os Conselheiros do Tribunal de Contas do Estado, à unanimidade, '
 'divergindo parcialmente do posicionamento do Corpo Instrutivo e do MPC – que '
 'opinaram pelo registro com fundamento diverso –, e com fulcro nos '
 'fundamentos dantes explanados, acolhendo integralmente o voto do Conselheiro '
 'Relator, julgar:\n'
 'a) pelo REGISTRO TÁCITO do ato de admissão em apreço; \n'
 'b) RECOMENDAR ao gestor responsável pela Prefeitura de Campos Grande/RN, que '
 'adote as providências necessárias ao saneamento da impropriedade apurada na '
 'Lei Municipal nº 233/2013, a fim de que passe a nela constar a nomenclatura '
 'do cargo de Técnico de Enfermagem, ao invés do cargo de Auxiliar de '
 'Enfermagem, na medida em que os requisitos exigidos pela mencionada Lei '
 'guardam relação com aquele cargo e não com este; \n'
 'c) pela remessa destes autos, após o trânsito em julgado deste Decisum, ao '
 'arquivo. \n'
 'Destaca, outrossim, que a apuração de responsabilidade pertinente ao E